In [1]:
import os

In [11]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality sixth\\end-to-end-wine-quality-fifth'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality sixth\\end-to-end-wine-quality-fifth'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    preprocessor_path: Path
    test_array_path: Path
    train_array_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

[2024-01-26 11:28:23,670: INFO: utils: NumExpr defaulting to 4 threads.]


In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEME_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            preprocessor_path=config.preprocessor_path,
            test_array_path = config.test_array_path,
            train_array_path=config.train_array_path
        )

        return data_transformation_config



In [8]:
import joblib
def save_object(file_path, obj):
    try:
        dir_path = os.path.dirname(file_path)
        os.makedirs(dir_path, exist_ok=True)

        with open(file_path, 'wb') as file_obj:
            joblib.dump(obj, file_obj)

    except Exception as e:
        raise e
def load_object(file_path):
    try:
        with open(file_path, 'rb') as file_obj:
            return joblib.load(file_obj)
    except Exception as e:
        raise e


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from mlProject import *
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np
# from mlProject.utils.common import save_bin, load_bin, save_object, load_object

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def get_data_transformation_object(self):
        ''' This function is responsible for data transformation'''
        try:
            numerical_columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
            categorical_columns = []
            num_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="median")),
                    ("scaler", StandardScaler(with_mean=False))
                ]
            )
            cat_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("one_hot_encoder", OneHotEncoder()),
                    ("scaler", StandardScaler(with_mean=False))
                ]
            )

            logging.info(f"Categorical columns : {categorical_columns}")
            logging.info(f"Numerical columns: {numerical_columns}")

            preprocessor = ColumnTransformer(
                [
                    ("num_pipeline", num_pipeline, numerical_columns),
                    ("cat_pipeline", cat_pipeline, categorical_columns)
                ]
            )

            return preprocessor
        except Exception as e:
            raise e
        
    def initiate_data_transformation(self):
        try:
            data = pd.read_csv(self.config.data_path)

            train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

            train_df.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
            test_df.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

            logging.info("Read Train and Test data completed")

            logging.info("Obtaining preprocessing object")
            preprocessor_obj = self.get_data_transformation_object()

            target_column_name = 'quality'

            input_feature_train_df = train_df.drop(columns=[target_column_name], axis=1)
            target_feature_train_df = train_df[target_column_name]

            input_feature_test_df = test_df.drop(columns=[target_column_name], axis=1)
            target_feature_test_df = test_df[target_column_name]

            logging.info("Applying preprocessing object on trianing and testing dataframe")

            input_feature_train_arr = preprocessor_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr = preprocessor_obj.transform(input_feature_test_df)

            train_arr = np.c_[input_feature_train_arr, np.array(target_feature_train_df)]
            test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]

            logging.info("Saved preprocessing object")

            save_object(file_path= Path(self.config.train_array_path), obj= train_arr)
            save_object(file_path= Path(self.config.test_array_path), obj=test_arr)

            logging.info('train and test array saved.')

        except Exception as e:
            raise e

    # def train_test_splitting(self):
    #     data = pd.read_csv(self.config.data_path)

    #     train, test = train_test_split(data, test_size=0.2, random_state=42)

    #     train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
    #     test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

    #     logger.info('Train test split started')
    #     logger.info(train.shape)
    #     logger.info(test.shape)

    #     print(train.shape)
    #     print(test.shape)

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.initiate_data_transformation()
except Exception as e:
    raise e

[2024-01-26 11:29:51,879: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-26 11:29:51,886: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-26 11:29:51,894: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-26 11:29:51,898: INFO: common: Created directory at artifacts]
[2024-01-26 11:29:51,901: INFO: common: Created directory at artifacts/data_transformation]


[2024-01-26 11:29:52,649: INFO: 1513097289: Read Train and Test data completed]
[2024-01-26 11:29:52,650: INFO: 1513097289: Obtaining preprocessing object]
[2024-01-26 11:29:52,654: INFO: 1513097289: Categorical columns : []]
[2024-01-26 11:29:52,656: INFO: 1513097289: Numerical columns: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]
[2024-01-26 11:29:52,722: INFO: 1513097289: Applying preprocessing object on trianing and testing dataframe]
[2024-01-26 11:29:52,855: INFO: 1513097289: Saved preprocessing object]
[2024-01-26 11:29:52,971: INFO: 1513097289: train and test array saved.]
